In [1]:
import queue
import time
import threading
import threaded_async

In [2]:
# Baseline (threads only)
N = 1000

def worker(in_queue, out_queue):
  for _ in range(N):
    x = in_queue.get()
    y = x + 1
    out_queue.put(y)

def main():
  in_queue = queue.Queue()
  out_queue = queue.Queue()

  workert = threading.Thread(target=worker, args=(in_queue, out_queue))
  workert.start()
  time.sleep(0.1)

  before = time.time()
  for i in range(N):
    in_queue.put(i)
    out_queue.get()
  after = time.time()

  print(f'Average: {(after - before) / N}')

main()

Average: 4.2430400848388674e-05


In [3]:
async def worker_async(in_queue, out_queue):
  for _ in range(N):
    x = await in_queue.get()
    y = x + 1
    await out_queue.put(y)

def main_async():
  with threaded_async.AsyncRunner() as runner:
    in_queue = threaded_async.Queue(runner)
    out_queue = threaded_async.Queue(runner)
    task = runner.create_task(worker_async(in_queue, out_queue))
    time.sleep(0.1)
    before = time.time()
    for i in range(N):
      in_queue.put_wait(i)
      out_queue.get_wait()
    after = time.time()
    task.cancel()
  print(f'Average: {(after - before) / N}')

main_async()

Average: 0.00028458237648010254
